# Let's do some biomechanics !!

Now, we are working with a arm model, actuated by muscles and pure joint torques (to compensate for model weakness). 
The goal is, starting from an initial position, to perform a reaching  task with this arm, by superimposing two markers. 
The first one (the target, 0th marker of the model), is fixed in the scene.
The second one (5th marker of the model) is fixed on the arm, in the *ulna* coordinate system.

The ocp to solve is as follow:
- Movement of $3s$ divided into $50$ shooting points
- The state variables are the generalized position (q) and velocities (qdot)
- The control variables are the muscle activations (a) 
- Cost functions: minimize the control effort at all time and minimize the Euclidean distance between the two markers at the final node
- Initial guess is arbitrarily set to $0.5$ for muscle activations and to $0$ for all other variables

In [ ]:
# So first, let's import all the required classes 
import biorbd
from bioptim import *

In [2]:
# Let's define the OCP
final_time= 3
n_shooting = 50

# Load the model and define some aliases
model = biorbd.Model("models/arm26.bioMod")
nq = model.nbQ()
ndq = model.nbQdot()
ntau = model.nbGeneralizedTorque()
nmus = model.nbMuscleTotal()

# Add objective functions
objective_functions = ObjectiveList()
objective_functions.add(ObjectiveFcn.Lagrange.MINIMIZE_TORQUE)
objective_functions.add(ObjectiveFcn.Lagrange.MINIMIZE_MUSCLES_CONTROL)
objective_functions.add(
    ObjectiveFcn.Mayer.SUPERIMPOSE_MARKERS, first_marker_idx=5, second_marker_idx=0, weight=1000
)

# Dynamics
dynamics = DynamicsList()
dynamics.add(DynamicsFcn.MUSCLE_ACTIVATIONS_AND_TORQUE_DRIVEN)

# Path constraint
x_bounds = BoundsList()
x_bounds.add(bounds=QAndQDotBounds(model))
x_bounds[0][:, 0] = (0.07, 1.4, 0, 0)

# Initial guess
x_init = InitialGuessList()
x_init.add([1.57] * nq + [0] * ndq)

# Define control path constraint
muscle_min, muscle_max, muscle_init = 0, 1, 0.5
tau_min, tau_max, tau_init = -1, 1, 0
u_bounds = BoundsList()
u_bounds.add(
    [tau_min] * ntau + [muscle_min] * nmus,
    [tau_max] * ntau + [muscle_max] * nmus,
)

u_init = InitialGuessList()
u_init.add([tau_init] * ntau + [muscle_init] * nmus)
# ------------- #

ocp = OptimalControlProgram(
    model,
    dynamics,
    n_shooting,
    final_time,
    x_init,
    u_init,
    x_bounds,
    u_bounds,
    objective_functions,
    n_threads = 8,
)


In [3]:
sol = ocp.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2584
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     3861

Total number of variables............................:      600
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      600
                     variables with only upper bounds:        0
Total number of equal

In [ ]:
sol.graphs(automatically_organize=False)

In [ ]:
sol.animate()

In [21]:
sol.print()

Solving time: 31.410675525665283 sec
Elapsed time: 31.410675525665283 sec

---- COST FUNCTION VALUES ----
PHASE 0
MINIMIZE_TORQUE: 21.4484514743373 (weighted 1.28691)
MINIMIZE_MUSCLES_CONTROL: 108.91822024087644 (weighted 6.53509)
SUPERIMPOSE_MARKERS: 0.0037584885614269014 (weighted 3.75849)

Sum cost functions: 11.5805
------------------------------

--------- CONSTRAINTS ---------
CONTINUITY 0: -5.7792248406990376e-08

PHASE 0

------------------------------


In [22]:
np.sqrt(0.003)

0.05477225575051661